<a href="https://colab.research.google.com/github/dominicwhat/Data-AI---RecSys/blob/main/sample_content_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 15:27:43--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.4’

All_Beauty.csv.4    100%[===================>]  14.78M  20.1MB/s    in 0.7s    

2022-01-02 15:27:44 (20.1 MB/s) - ‘All_Beauty.csv.4’ saved [15499476/15499476]

--2022-01-02 15:27:44--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.4’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.2MB/s    in 0.6s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

以2018-09-01為分界，切training data和testing data

In [7]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

僅考慮testing data前一年同月份的資料作為training data

In [8]:
ratings_trainings_last_sep =  ratings_trainings[
    (ratings_trainings['DATE'] >= '2017-09-01') & 
    (ratings_trainings['DATE'] <= '2017-09-30')
]
asin_group_last_sep = ratings_trainings_last_sep.groupby(by=['asin']).agg(list).reset_index()
asin_group_last_sep['interaction_num'] = [len(i) for i in asin_group_last_sep['overall']]
asin_group_last_sep = asin_group_last_sep.sort_values(by='interaction_num', ascending=False,ignore_index = True)
asin_group_last_sep.head()

,asin,reviewerID,overall,unixReviewTime,DATE,interaction_num
0,B0010ZBORW,"[A1BTG38XSGYLW3, A3S3R88HA0HZG3, A3SLC8F6VIWXI...","[5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 3.0, 5.0, 4.0, ...","[1506729600, 1506729600, 1506643200, 150664320...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",400
1,B00W259T7G,"[A1RF9YK4BK5TRH, A2CNGARED6OWWU, A30Z0GG96D11E...","[3.0, 4.0, 3.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","[1506729600, 1506729600, 1506729600, 150672960...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",387
2,B01DKQAXC0,"[A1U73UEOOKPCAG, A3MTSOTN6KI5YQ, A3LHIPR8FUS9B...","[1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, ...","[1506729600, 1506729600, 1506556800, 150655680...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",124
3,B000050FDY,"[AEULHIYYGAGE2, A4QU03PP6IKZ, A3O00BUEBIFVAR, ...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[1506729600, 1506729600, 1506729600, 150672960...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",103
4,B001LNODUS,"[A16WPA6JV83YXT, A10ZBR6O8S8OCY, A9P07NJ7UV0M,...","[2.0, 4.0, 2.0, 5.0, 3.0, 3.0, 4.0, 5.0, 5.0, ...","[1506729600, 1506643200, 1506556800, 150647040...","[2017-09-30 00:00:00, 2017-09-29 00:00:00, 201...",87


In [9]:
#ratings_trainings_unique = ratings_trainings['reviewerID'].unique()
#sum([sum(ratings_trainings_unique == user) for user in users])

## 產生推薦

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算商品用標題所表示的 tfidf 矩陣
df = metadata.drop_duplicates('title')
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(df['title'])

# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦(Content-based)
def recommend_items_content(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res


# 利用使用者購買過的商品產生推薦(Content-based & Ruled-based)
def recommend_items_content_rule(items, k):
    res = []
    if len(items) != 0:
      for d in items:
        res.extend(recommend_item(d, k))
      return res
    else:
      res.extend(asin_group_last_sep['asin'][0:k].tolist())
      return res 



In [11]:
def recommender(training_data, users=[], k=10, rule = False):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    if rule:
      recommendations = {user: recommend_items_content_rule(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}
    else:
      recommendations = {user: recommend_items_content(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}
    return recommendations

ratings_by_user_content = recommender(ratings_trainings, users, rule=False)
ratings_by_user_content
ratings_by_user_content_rule = recommender(ratings_trainings, users, rule=True)
ratings_by_user_content_rule

{'A100XQFWKQ30O2': ['B0010ZBORW',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000050FDY',
  'B001LNODUS',
  'B019FWRG3C',
  'B001QY8QXM',
  'B002GP80EU',
  'B0195R1FT8',
  'B01BZV6VVY'],
 'A103T1QOGFCSEH': ['B0010ZBORW',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000050FDY',
  'B001LNODUS',
  'B019FWRG3C',
  'B001QY8QXM',
  'B002GP80EU',
  'B0195R1FT8',
  'B01BZV6VVY'],
 'A106UKKSJ2KXPF': ['B0010ZBORW',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000050FDY',
  'B001LNODUS',
  'B019FWRG3C',
  'B001QY8QXM',
  'B002GP80EU',
  'B0195R1FT8',
  'B01BZV6VVY'],
 'A10A7GV4D5A11V': ['B0010ZBORW',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000050FDY',
  'B001LNODUS',
  'B019FWRG3C',
  'B001QY8QXM',
  'B002GP80EU',
  'B0195R1FT8',
  'B01BZV6VVY'],
 'A1119JJ37ZLB8R': ['B0010ZBORW',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000050FDY',
  'B001LNODUS',
  'B019FWRG3C',
  'B001QY8QXM',
  'B002GP80EU',
  'B0195R1FT8',
  'B01BZV6VVY'],
 'A113UOOLBSZN52': ['B0010ZBORW',
  'B00W259T7G',
  'B01DKQAXC0',
  'B000050FDY',
  'B001LNODUS',
  'B019

## 結果評估

In [12]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

print("default_content_based",evaluate(ratings_testings_by_user, ratings_by_user_content))
print("content_ruled_based",evaluate(ratings_testings_by_user, ratings_by_user_content_rule))

default_content_based 0.0
content_ruled_based 0.09661016949152543
